In [15]:
import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np

In [16]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [17]:
#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
#Linux
#os.chdir('/home/kwnabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models
finbert_models = config.finbert_models
database = config.Local_Database

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
# Model_Folder = config.Sentiment_models
# Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2


/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [18]:
df1 = pd.read_csv(f"{finbert_models}/{Body}/{Model}/{Body}_{Model}_finbert_model_short.csv")
df2 = pd.read_csv(f"{finbert_models}/{Body_2}/{Model_2}/{Body_2}_{Model_2}_finbert_model_short.csv")

In [19]:


df1 = df1[['date', 'sentiment']]

df1 = df1.groupby(['date']).mean().reset_index()
df1['date'] = pd.to_datetime(df1['date'])
df1['event'] = 1
dfb = df1.copy(deep=True)
dfb = dfb[['date']]
dfb['before'] = 1
dfa = df1.copy(deep=True)
dfa = dfa[['date']]
dfa['after'] = 1
dfb['date'] = dfb['date'] - pd.to_timedelta(1, unit='d')
dfa['date'] = dfa['date'] + pd.to_timedelta(1, unit='d')  
df1 = pd.concat([dfa, df1, dfb])
df1 = df1.sort_values(by='date')


In [20]:
ffr = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/FFRF/FFRF.csv')
#Convert Date to datetime period days 
ffr['Date'] = pd.to_datetime(ffr['Date'])
ffr = ffr[['Date', 'Price', 'Vol.']]
#Rename Pirce to ffr_price and Vol. to ffr_vol
ffr = ffr.rename(columns={'Price': 'ffr_price', 'Vol.': 'ffr_vol'})
#on ffr_vol remove the k at the end of the number and convert to float
ffr['ffr_vol'] = ffr['ffr_vol'].str.replace('K', '')
ffr['ffr_vol'] = ffr['ffr_vol'].astype(float)
ffr['ffr_vol'] = ffr['ffr_vol'] * 1000

print(ffr.head())

tb10y = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/10Y Treasury/T10YFF.csv')

tb10y['DATE'] = pd.to_datetime(tb10y['DATE'])
tb10y = tb10y.rename(columns={'T10YFF': 'tb10y'})

tb10y = tb10y[tb10y['tb10y'] != '.']
print(tb10y.head())

ir1 = pd.read_csv(f"{database}/Other Data/2YIR.csv")
ir1 = ir1.rename(columns={'DATE': 'date', 'DGS2': 'IR2'})
ir1['date'] = pd.to_datetime(ir1['date'])
ir1 = ir1[ir1['IR2'] != '.']

        Date  ffr_price    ffr_vol
0 2019-11-14    98.4400 18080.0000
1 2019-11-13    98.4400 48770.0000
2 2019-11-12    98.4300 27110.0000
3 2019-11-11    98.4300  2380.0000
4 2019-11-08    98.4300 32870.0000
        DATE tb10y
0 1962-01-02  1.31
1 1962-01-03  1.53
2 1962-01-04  1.24
3 1962-01-05  1.52
4 1962-01-08  2.03


In [21]:
#Convert date to datetime
df1['date'] = pd.to_datetime(df1['date'])
df1 = df1.merge(ffr, how='outer', left_on='date', right_on='Date')
df1 = df1.drop(columns=['Date'])

df1 = df1.merge(tb10y, how='outer', left_on='date', right_on='DATE')
df1 = df1.drop(columns=['DATE'])

df1 = df1.merge(ir1, how='outer', left_on='date', right_on='date')

#Convert tb10y and IR2 to float
df1['tb10y'] = df1['tb10y'].astype(float)
df1['IR2'] = df1['IR2'].astype(float)

print(df1.dtypes)
#df1 = df1.dropna()
print(df1.head(-10))   


date         datetime64[ns]
after               float64
sentiment           float64
event               float64
before              float64
ffr_price           float64
ffr_vol             float64
tb10y               float64
IR2                 float64
dtype: object
            date  after  sentiment  event  before  ffr_price    ffr_vol  \
0     2000-02-01    NaN        NaN    NaN  1.0000    94.1900  3750.0000   
1     2000-02-02    NaN     0.2857 1.0000     NaN    94.2400 12560.0000   
2     2000-02-03 1.0000        NaN    NaN     NaN    94.2500  7190.0000   
3     2000-03-20    NaN        NaN    NaN  1.0000    94.1400  1420.0000   
4     2000-03-21    NaN     0.1429 1.0000     NaN    94.1600  1720.0000   
...          ...    ...        ...    ...     ...        ...        ...   
28923 2023-12-14    NaN        NaN    NaN     NaN        NaN        NaN   
28924 2023-12-15    NaN        NaN    NaN     NaN        NaN        NaN   
28925 2023-12-18    NaN        NaN    NaN     NaN        Na

In [22]:
df_temp = pd.read_csv(f"/Users/kylenabors/Documents/Database/Other Data/SPY/SPYs.csv")

In [23]:
spy = df_temp.copy(deep=True)
spy = spy[['DATE', 'CRSPTM1']]
spy = spy.rename(columns={'DATE': 'date', 'CRSPTM1': 'spy_price'})
spy["date"] = pd.to_datetime(spy["date"])
spy = spy.groupby(['date']).mean().reset_index()
print(spy.head()) 


        date  spy_price
0 2011-04-01  1006.5733
1 2011-04-04  1005.2782
2 2011-04-05  1007.8674
3 2011-04-06  1007.7223
4 2011-04-07  1005.8883


In [24]:

spy = pd.merge(spy, df1, how='outer', left_on='date', right_on='date')
#spy = spy.drop(columns=['date_y'])
#spy = spy.rename(columns={'date_x': 'date'})
#spy = spy[['date', 'spy_price', 'sentiment', 'time']]
#spy = spy.dropna()


spy["positive"] = np.where(spy["sentiment"] > 0.05, 1, 0)
spy["negative"] = np.where(spy["sentiment"] < -0.05, 1, 0)
spy["neutral"] = np.where((spy["sentiment"] <= 0.05) & (spy["sentiment"] >= -0.05), 1, 0)
print(spy.head())

spy_pos = spy[spy['positive'] == 1]
spy_neg = spy[spy['negative'] == 1]
spy_neu = spy[spy['neutral'] == 1]

        date  spy_price  after  sentiment  event  before  ffr_price  ffr_vol  \
0 2011-04-01  1006.5733    NaN        NaN    NaN     NaN        NaN      NaN   
1 2011-04-04  1005.2782    NaN        NaN    NaN     NaN        NaN      NaN   
2 2011-04-05  1007.8674    NaN        NaN    NaN     NaN        NaN      NaN   
3 2011-04-06  1007.7223    NaN        NaN    NaN     NaN        NaN      NaN   
4 2011-04-07  1005.8883    NaN        NaN    NaN     NaN        NaN      NaN   

   tb10y    IR2  positive  negative  neutral  
0    NaN 0.8000         0         0        0  
1    NaN 0.7700         0         0        0  
2    NaN 0.8400         0         0        0  
3    NaN 0.8500         0         0        0  
4    NaN 0.8100         0         0        0  


In [25]:
dfr = spy[['date', 'spy_price', 'sentiment', 'ffr_price', 'tb10y', 'IR2', 'event', 'before', 'after']]
dfr = dfr.groupby(['date']).mean().reset_index()
dfr = dfr[(dfr['event'] == 1) | (dfr['before'] == 1) | (dfr['after'] == 1)]
print(dfr.head())

           date  spy_price  sentiment  ffr_price  tb10y    IR2  event  before  \
2523 2000-02-01        NaN        NaN    94.1900 0.8300 6.6000    NaN  1.0000   
2524 2000-02-02        NaN     0.2857    94.2400 0.9600 6.6300 1.0000     NaN   
2525 2000-02-03        NaN        NaN    94.2500 0.7800 6.5600    NaN     NaN   
2556 2000-03-20        NaN        NaN    94.1400 0.3600 6.5200    NaN  1.0000   
2557 2000-03-21        NaN     0.1429    94.1600 0.3200 6.5000 1.0000     NaN   

      after  
2523    NaN  
2524    NaN  
2525 1.0000  
2556    NaN  
2557    NaN  
